<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#NFL-PxP-Data" data-toc-modified-id="NFL-PxP-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>NFL PxP Data</a></span><ul class="toc-item"><li><span><a href="#Extract-Kickoffs-and-Possession-Starts" data-toc-modified-id="Extract-Kickoffs-and-Possession-Starts-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Extract Kickoffs and Possession Starts</a></span></li><li><span><a href="#Extract-Punts" data-toc-modified-id="Extract-Punts-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Extract Punts</a></span></li><li><span><a href="#Extract-Field-Goals" data-toc-modified-id="Extract-Field-Goals-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Extract Field Goals</a></span></li><li><span><a href="#Extract-Third-Downs" data-toc-modified-id="Extract-Third-Downs-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Extract Third Downs</a></span></li></ul></li></ul></div>

In [1]:
import numpy as np
import pandas as pd

# Extracting Plays

This notebook is an optional companion to the homework and shows how the play-by-play data was handled to extract the necessary components for modeling 4th downs.

## NFL PxP Data

NFL play-by-play data is loaded from csv format.  

Below is a list of avaible columns we will use.  Many are self explanatory so when needed, a description will be given.  Note, there are many more fields available that we will not use.
+ GameID
+ Drive - index given the # of the drive within the game
+ qtr
+ down
+ yrdline100 - the yard line expressed on a scale of 1 to 99 instead of 1 to 50 and back to 1.
+ ydstogo - yards to go for a first down
+ Yards.Gained - yards gained on the play
+ posteam - possessing team
+ DefensiveTeam - defensive team
+ ~~desc - play description~~  <-- Had to drop this due to memory problems
+ PlayType - label for what type of play
+ Touchdown - 0,1 indicating if a TD was scored
+ FieldGoalResult - label indicating good, blocked, or no good.
+ FieldGoalDistance
+ PosTeamScore - Score of the possessing team.  This will flip when the possession flips.
+ DefTeamScore - Score of the defensive team.  This will flip when the possession flips.
+ HomeTeam
+ AwayTeam


A few convenient data fields are added to easy computation of possession value.

+ half
+ yrdregion - region of the field: Inside the 10, 10 to 20, and beyond 20.
+ HomeScore & AwayScore - The score of the possession and defensive teams are given.  This changes as the ball changes possession
+ nextposteam - The team possessing the ball in the next play. Non-plays are ignored
+ nextyrdline100 - Where the ball is on the next play. Non-plays are ignored
+ nextdown - The down for the next play
+ 1stdownconversion - Whether the current play converted a first down (0 or 1 value)

In [2]:
pxp = pd.read_csv('data/nfl_pxp_2009_2016.csv.gz')

In [3]:
pxp.head()

,GameID,Drive,qtr,down,yrdline100,ydstogo,posteam,DefensiveTeam,Yards.Gained,Touchdown,...,HomeTeam,AwayTeam,half,yrdregion,HomeScore,AwayScore,nextposteam,nextyrdline100,nextdown,1stdownconversion
0,2009091000,1,1,NaN,30.0,0,PIT,TEN,39,0,...,PIT,TEN,1,Beyond20,0.0,0.0,PIT,58.0,1.0,1
1,2009091000,1,1,1.0,58.0,10,PIT,TEN,5,0,...,PIT,TEN,1,Beyond20,0.0,0.0,PIT,53.0,2.0,0
2,2009091000,1,1,2.0,53.0,5,PIT,TEN,-3,0,...,PIT,TEN,1,Beyond20,0.0,0.0,PIT,56.0,3.0,0
3,2009091000,1,1,3.0,56.0,8,PIT,TEN,0,0,...,PIT,TEN,1,Beyond20,0.0,0.0,PIT,56.0,4.0,0
4,2009091000,1,1,4.0,56.0,8,PIT,TEN,0,0,...,PIT,TEN,1,Beyond20,0.0,0.0,TEN,98.0,1.0,1


### Extract Kickoffs and Possession Starts

We need to extract kickoffs and possession starts in order to build a possession value calculator.  To do that, we follow this process:
1. **Extract kickoffs by using PlayType.**
2. **Extract possession starts**  Drill down by GameID and Drive # (ignoring kickoffs) and take the first play of the drive.  We need to ignore kickoffs and extra points because those can both appear as the first play of a drive.  This will pollute our results if we do not ignore them.
    
3. **Find the next score in the game for each possession.**  This is the hardest computation.  We do this by computing differences in the home and away scores and then fill those backward.  We treat home scores as positive and away scores as negative.  We only consider possession value within a half.  So if there is no score before halftime or the end of the game, the value is 0.

4. **Compute possession value.**  We multiply the next score value by +1 or -1 depending on if the current possessing team is the home team or away team.  If its the home team, then multiply by +1 because the next score is already oriented to the home team.  If its the away team, then multiply by -1 because a positive next score is a negative for the away team.
5. **Restrict possessions to the first and third quarter.**  We want to avoid end of half/game effects like settling for points at the end of the first half or playing to win at the end of the game.

Some caveats:
+ The dataset used is not perfect so while we expect this procedure to work a vast majority of the time, it may miss some results because of holes in the dataset.  It is unlikely this affects the analysis too much.
+ While we restricted to first and third quarters, we did not restrict cases when there is a blowout.  Competitive games lead to more reliable results so this is probably the first issue to address going forward.

In [4]:
def extract_kickoffs_and_possessions(pxp):
    # Step 1: extract kickoffs
    ko_mask = pxp['PlayType'] == 'Kickoff'
    kickoffs = pxp.loc[ko_mask]

    # Step 2: extracting possessions
    # Exclude kickoffs and XPs and groupby GameID and Drive
    xp_mask = pxp['PlayType'] == 'Extra Point'
    game_drives = pxp.loc[~ko_mask & ~xp_mask].groupby(['GameID', 'Drive'])

    # Take first play
    poss_starts = game_drives.head(1)

    # Concatenate kickoffs and first plays, sort, and reindex
    poss = pd.concat([kickoffs, poss_starts])
    poss.sort_values(['GameID', 'Drive', 'down'],
                     na_position='first', inplace=True)
    poss.reset_index(drop=True, inplace=True)

    # Step 3: Find the next score
    # Group by game halves
    game_halves = poss.groupby(['GameID', 'half'])
    # Compute changes in the scores.  + for Home and - for Away.
    score_change = game_halves['AwayScore'].diff(-1) - \
        game_halves['HomeScore'].diff(-1)
    # Backfill the score change so that each possession now has a value for next score in the game
    next_score = score_change.replace(to_replace=0., method='bfill').fillna(0)
    poss['NextScore'] = next_score

    # Step 4: Compute possession value
    # Determine if the possessing team is home or away
    posteam = poss['posteam']
    hometeam = poss['HomeTeam']
    awayteam = poss['AwayTeam']
    posteam_is_home = np.where(posteam == hometeam, 1, 0)
    posteam_is_away = np.where(posteam == awayteam, 1, 0)
    # NextScore is unchanged if posteam == hometeam and negated if posteam == awayteam
    poss['PossessionValue'] = poss['NextScore'] * \
        (posteam_is_home - posteam_is_away)

    # Step 5: Retrict to first and third quarters
    first_and_third_qtr = (poss['qtr'] == 1) | (poss['qtr'] == 3)
    poss = poss.loc[first_and_third_qtr].copy()

    cols = ['GameID', 'HomeTeam', 'AwayTeam', 'Drive', 'half', 'qtr', 'down',  'posteam',
            'yrdline100', 'yrdregion', 'PlayType', 'NextScore', 'PossessionValue']
    poss = poss[cols]
    poss.reset_index(drop=True, inplace=True)
    return poss

In [5]:
poss = extract_kickoffs_and_possessions(pxp)
poss.to_csv('data/possessions.csv', index=False)
poss.head(20)

,GameID,HomeTeam,AwayTeam,Drive,half,qtr,down,posteam,yrdline100,yrdregion,PlayType,NextScore,PossessionValue
0,2009091000,PIT,TEN,1,1,1,NaN,PIT,30.0,Beyond20,Kickoff,7.0,7.0
1,2009091000,PIT,TEN,1,1,1,1.0,PIT,58.0,Beyond20,Pass,7.0,7.0
2,2009091000,PIT,TEN,2,1,1,1.0,TEN,98.0,Beyond20,Run,7.0,-7.0
3,2009091000,PIT,TEN,3,1,1,1.0,PIT,43.0,Beyond20,Pass,7.0,7.0
4,2009091000,PIT,TEN,4,1,1,1.0,TEN,89.0,Beyond20,Run,7.0,-7.0
5,2009091000,PIT,TEN,5,1,1,1.0,PIT,73.0,Beyond20,Run,7.0,7.0
6,2009091000,PIT,TEN,6,1,1,1.0,TEN,74.0,Beyond20,Pass,7.0,-7.0
7,2009091000,PIT,TEN,7,1,1,1.0,PIT,79.0,Beyond20,Run,7.0,7.0
8,2009091000,PIT,TEN,8,1,1,1.0,TEN,44.0,Beyond20,Run,7.0,-7.0
9,2009091000,PIT,TEN,14,2,3,NaN,TEN,30.0,Beyond20,Kickoff,-3.0,3.0


### Extract Punts

We need to extract net punt distance.  Extracing punts is easy.  Determining net punt distance is tricky.  A few things can happen on a punt: 
+ It goes off as expected
+ Returned for a touchdown
+ Muffed
+ Fumbled
+ Blocked.  

There may be even more wrinkles.  Suffice to say, this is a bit complicated.  To simplify things, we just want to know the expected net punt distance for punts that are not returned, muffed, fumbled, blocked, or anything else.  

Why is it okay to simplify things?  Those complicating events do not happen often enough to materially affect the computation and just add a nuisance to the whole model.  Consider a return TD worth about 7 points.  If a return TD happens every 100 punts, then its worth about 0.01 points in expectation.  Not really a huge amount.

We compute net punt distance by finding regular punts and determining the field position change.

In [6]:
def extract_punts(pxp):
    # Build table of punts    
    punt_mask = pxp['PlayType'] == 'Punt'
    punts = pxp.loc[punt_mask].copy()
    punts.reset_index(drop=True, inplace=True)

    # Determine if there was a possession change, ie. the punt went off as expected.
    # This excludes muffs or fumbled returns.  It also exludes return TDs and probably
    # some other cases.  It is likely not too bad to do this since these events are rare.
    posteam = punts['posteam']
    nextposteam = punts['nextposteam']
    punts = punts.loc[(posteam != nextposteam)].copy()

    # Determine the net punt distance
    yrdline = punts['yrdline100']
    nextyardline = punts['nextyrdline100'] 
    net_punt_dist = (yrdline - (100 - nextyardline))

    # Add net punt length to punts table
    punts['net_punt_dist'] = net_punt_dist

    cols = ['GameID', 'HomeTeam', 'AwayTeam', 'Drive', 'half', 'qtr', 'down',  'posteam',
            'yrdline100', 'yrdregion', 'PlayType', 'net_punt_dist']
    punts = punts[cols]
    punts.reset_index(drop=True, inplace=True)
    return punts

In [7]:
punts = extract_punts(pxp)
punts.to_csv('data/punts.csv', index=False)
punts.head(10)

,GameID,HomeTeam,AwayTeam,Drive,half,qtr,down,posteam,yrdline100,yrdregion,PlayType,net_punt_dist
0,2009091000,PIT,TEN,1,1,1,4.0,PIT,56.0,Beyond20,Punt,54.0
1,2009091000,PIT,TEN,2,1,1,4.0,TEN,96.0,Beyond20,Punt,39.0
2,2009091000,PIT,TEN,3,1,1,4.0,PIT,41.0,Beyond20,Punt,30.0
3,2009091000,PIT,TEN,5,1,1,4.0,PIT,79.0,Beyond20,Punt,53.0
4,2009091000,PIT,TEN,8,1,2,4.0,TEN,44.0,Beyond20,Punt,39.0
5,2009091000,PIT,TEN,9,1,2,4.0,PIT,62.0,Beyond20,Punt,35.0
6,2009091000,PIT,TEN,15,2,3,4.0,PIT,45.0,Beyond20,Punt,40.0
7,2009091000,PIT,TEN,16,2,3,4.0,TEN,96.0,Beyond20,Punt,50.0
8,2009091000,PIT,TEN,17,2,3,4.0,PIT,43.0,Beyond20,Punt,33.0
9,2009091000,PIT,TEN,18,2,3,4.0,TEN,71.0,Beyond20,Punt,56.0


### Extract Field Goals

For field goal results, we just need FieldGoalDistance and FieldGoalResult and then to compute a 0-1 value representing the success.  This is straightforward.

When it comes to making 4th down decisions, the quality of the kicker and the field/weather conditions should definitely be taken into consideration.  Remember, we are computing a baseline "average" model that can be used as a starting point.

In [8]:
def extract_fgs(pxp):
    # Extract field goals
    fg_mask = (pxp['PlayType'] == 'Field Goal')
    fg_cols = ['FieldGoalDistance', 'FieldGoalResult']
    fgs = pxp.loc[fg_mask, fg_cols]
    # restrict to fgs less than or equal to 63 yards
    fgs = fgs.loc[fgs['FieldGoalDistance'] <= 63.].copy()
    # Compute success flag
    fgs['FieldGoalSuccess'] = np.where(fgs['FieldGoalResult'] == 'Good', 1, 0)
    
    fgs.reset_index(drop=True, inplace=True)
    return fgs

In [9]:
fgs = extract_fgs(pxp)
fgs.to_csv('data/fgs.csv', index=False)
fgs.head(20)

,FieldGoalDistance,FieldGoalResult,FieldGoalSuccess
0,37.0,No Good,0
1,31.0,Blocked,0
2,45.0,Good,1
3,32.0,Good,1
4,33.0,Good,1
5,37.0,Good,1
6,21.0,Good,1
7,20.0,Good,1
8,37.0,Good,1
9,47.0,Good,1


### Extract Third Downs

First, why third downs?  Since most teams do not go for it on fourth down, we need a proxy for the likelihood of converting the first down.  Since third downs are generally considered make-or-break, we opt to substitute performance on third downs for fourth down.

In the dataset, various non-relevant plays can occur on third down.  We need to ignore these.  Also, we can likely safely assume that beyond 9 yards we do not need to consider going for it.

In [10]:
def extract_third_downs(pxp):
    # Ignore certain set of plays
    ignored_plays = ['Punt', 'Field Goal', 'No Play', 'QB Kneel', 'Spike']
    ignored_plays_mask = pxp['PlayType'].isin(ignored_plays)
    valid_plays = pxp.loc[~ignored_plays_mask]
    # third downs
    all_third_downs_mask = (valid_plays['down'] == 3)
    # Restrict to less than 10 yards to go
    under_10_to_go = (valid_plays['ydstogo'] <= 9)
    third_down_mask = all_third_downs_mask & under_10_to_go
    # Extract relevant plays
    third_down_plays = valid_plays.loc[third_down_mask].copy()
    
    cols = ['GameID', 'HomeTeam', 'AwayTeam', 'Drive', 'half', 'qtr', 'down',  'posteam',
            'yrdline100', 'yrdregion', 'ydstogo', 'PlayType', '1stdownconversion']
    third_down_plays = third_down_plays[cols]
    third_down_plays.reset_index(drop=True, inplace=True)
    return third_down_plays

In [11]:
third_downs = extract_third_downs(pxp)
third_downs.to_csv('data/third_downs.csv', index=False)
third_downs.head(20)

,GameID,HomeTeam,AwayTeam,Drive,half,qtr,down,posteam,yrdline100,yrdregion,ydstogo,PlayType,1stdownconversion
0,2009091000,PIT,TEN,1,1,1,3.0,PIT,56.0,Beyond20,8,Pass,0
1,2009091000,PIT,TEN,2,1,1,3.0,TEN,94.0,Beyond20,6,Run,0
2,2009091000,PIT,TEN,3,1,1,3.0,PIT,22.0,Beyond20,2,Sack,0
3,2009091000,PIT,TEN,4,1,1,3.0,TEN,19.0,10to20,7,Pass,0
4,2009091000,PIT,TEN,5,1,1,3.0,PIT,70.0,Beyond20,7,Sack,0
5,2009091000,PIT,TEN,6,1,1,3.0,TEN,57.0,Beyond20,3,Pass,1
6,2009091000,PIT,TEN,7,1,1,3.0,PIT,76.0,Beyond20,7,Pass,1
7,2009091000,PIT,TEN,9,1,2,3.0,PIT,87.0,Beyond20,2,Pass,1
8,2009091000,PIT,TEN,9,1,2,3.0,PIT,62.0,Beyond20,5,Pass,0
9,2009091000,PIT,TEN,10,1,2,3.0,TEN,69.0,Beyond20,6,Pass,1
